# Crawling
The two following print statements are just to add content to this notebook so something can be uploaded to Github and then reuploaded. I wanted to see how the web interface handles pushes from my machine. Hint: Refresh the page following a push.

In [1]:
print('Hello world!')

Hello world!


In [2]:
print('This is a git commit test.')

This is a git commit test.


# Standing
This section is to document the cleaning steps for the data files in the dataset folder. This duplicates my cleaning steps I performed in Excel for the Coursera Case Study 1. The report and processed dataset for that study can be found on my [Kaggle site](https://www.kaggle.com/code/scwilso28/coursera-case-study-1-cyclistic-data-analysis). A sample of the data has been uploaded here as a zip file to reduce file size.

In [3]:
import pandas as pd
import numpy as np
import zipfile

In [4]:
zf = zipfile.ZipFile('Datasets/202103-divvy-tripdata.zip')
df = pd.read_csv(zf.open('202103-divvy-tripdata.csv'))

## Memory usage
Following the data import and loading into a dataframe, the dataframe structure is reviewed. My goal of this next section is to reduce the dataframe size, as it was consuming 146.0 MB. This data and it's size on it's own isn't an issue, but I will be performing the cleanup on all of the data files (13 files) associated with the case study and this dataset is one of the smaller sets. By the way, the info method for data frames is very useful, here's the [manual](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.info.html).

In [5]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228496 entries, 0 to 228495
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ride_id             228496 non-null  object 
 1   rideable_type       228496 non-null  object 
 2   started_at          228496 non-null  object 
 3   ended_at            228496 non-null  object 
 4   start_station_name  213648 non-null  object 
 5   start_station_id    213648 non-null  object 
 6   end_station_name    211769 non-null  object 
 7   end_station_id      211769 non-null  object 
 8   start_lat           228496 non-null  float64
 9   start_lng           228496 non-null  float64
 10  end_lat             228329 non-null  float64
 11  end_lng             228329 non-null  float64
 12  member_casual       228496 non-null  object 
dtypes: float64(4), object(9)
memory usage: 146.0 MB


In [6]:
to_drop = ['start_station_id', 'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng']
df.drop(to_drop, inplace = True, axis = 1)

In [7]:
df['started_at'] = pd.to_datetime(df['started_at'])
df['ended_at'] = pd.to_datetime(df['ended_at'])

In [8]:
df['member_casual'] = df['member_casual'].map({'casual':0, 'member':1})
df.info(memory_usage="deep")
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228496 entries, 0 to 228495
Data columns (total 7 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   ride_id             228496 non-null  object        
 1   rideable_type       228496 non-null  object        
 2   started_at          228496 non-null  datetime64[ns]
 3   ended_at            228496 non-null  datetime64[ns]
 4   start_station_name  213648 non-null  object        
 5   end_station_name    211769 non-null  object        
 6   member_casual       228496 non-null  int64         
dtypes: datetime64[ns](2), int64(1), object(4)
memory usage: 69.9 MB


,ride_id,rideable_type,started_at,ended_at,start_station_name,end_station_name,member_casual
0,CFA86D4455AA1030,classic_bike,2021-03-16 08:32:30,2021-03-16 08:36:34,Humboldt Blvd & Armitage Ave,Stave St & Armitage Ave,0
1,30D9DC61227D1AF3,classic_bike,2021-03-28 01:26:28,2021-03-28 01:36:55,Humboldt Blvd & Armitage Ave,Central Park Ave & Bloomingdale Ave,0
2,846D87A15682A284,classic_bike,2021-03-11 21:17:29,2021-03-11 21:33:53,Shields Ave & 28th Pl,Halsted St & 35th St,0
3,994D05AA75A168F2,classic_bike,2021-03-11 13:26:42,2021-03-11 13:55:41,Winthrop Ave & Lawrence Ave,Broadway & Sheridan Rd,0
4,DF7464FBE92D8308,classic_bike,2021-03-21 09:09:37,2021-03-21 09:27:33,Glenwood Ave & Touhy Ave,Chicago Ave & Sheridan Rd,0


## Data cleaning
I had performed this study using Excel, so I have some knowledge of issues that plague this data. Specifically, there are duplicates and invalid ride durations. 

The duplicates are a little sneaky. When this data was generated, any personal information was removed and replaced with a unique ride_id value. Unfortunately, that means if one user somehow made multiple copies of one ride, it would register as two different ride_id's with equal information for the remaining columns. The approach to get around the unique ride_id is to check for duplicates across the rest of the columns. Without input from the bike share company or stakeholders, I decided to delete the duplicates and report the number deleted for documentation. I wouldn't advise doing this without additional input.

There are a few instances where the ended_at datetime is before the started_at datetime. I don't know how this happened, so again in isolation, the decision was made to just zero out the negative time deltas calculated to make this exercise work. Again, I don't recommend doing this right away. At least talk to someone before overriding values. Maybe the rider was a time traveler.

In [9]:
duplicate_check = ['rideable_type', 'started_at', 'ended_at', 'start_station_name', 'end_station_name', 'member_casual']
duplicate_entries = df[df.duplicated(subset = duplicate_check, keep = False)]
duplicate_list_length = len(duplicate_entries.drop_duplicates(subset=duplicate_check))
print(duplicate_list_length)

26


In [10]:
print(100*duplicate_list_length/len(df))

0.011378754989146419


In [11]:
print(len(df))

228496


In [12]:
df.drop_duplicates(subset = duplicate_check, inplace=True)
print(len(df))

228470


In [13]:
df['ride_duration'] = df.ended_at - df.started_at
df[['started_at', 'ended_at', 'ride_duration']].head()

,started_at,ended_at,ride_duration
0,2021-03-16 08:32:30,2021-03-16 08:36:34,0 days 00:04:04
1,2021-03-28 01:26:28,2021-03-28 01:36:55,0 days 00:10:27
2,2021-03-11 21:17:29,2021-03-11 21:33:53,0 days 00:16:24
3,2021-03-11 13:26:42,2021-03-11 13:55:41,0 days 00:28:59
4,2021-03-21 09:09:37,2021-03-21 09:27:33,0 days 00:17:56


In [14]:
print(df.ride_duration.min())

-1 days +23:59:59


In [15]:
zero_timedelta = pd.to_timedelta(0)
column = df['ride_duration']
print(column[column < zero_timedelta].count())

2


In [16]:
df['ride_duration'].values[df['ride_duration'].values < zero_timedelta] = zero_timedelta
print(column[column < zero_timedelta].count())

0


In [17]:
df['ride_duration_hrs'] = df.ride_duration / np.timedelta64(1, 'h')
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,end_station_name,member_casual,ride_duration,ride_duration_hrs
0,CFA86D4455AA1030,classic_bike,2021-03-16 08:32:30,2021-03-16 08:36:34,Humboldt Blvd & Armitage Ave,Stave St & Armitage Ave,0,0 days 00:04:04,0.067778
1,30D9DC61227D1AF3,classic_bike,2021-03-28 01:26:28,2021-03-28 01:36:55,Humboldt Blvd & Armitage Ave,Central Park Ave & Bloomingdale Ave,0,0 days 00:10:27,0.174167
2,846D87A15682A284,classic_bike,2021-03-11 21:17:29,2021-03-11 21:33:53,Shields Ave & 28th Pl,Halsted St & 35th St,0,0 days 00:16:24,0.273333
3,994D05AA75A168F2,classic_bike,2021-03-11 13:26:42,2021-03-11 13:55:41,Winthrop Ave & Lawrence Ave,Broadway & Sheridan Rd,0,0 days 00:28:59,0.483056
4,DF7464FBE92D8308,classic_bike,2021-03-21 09:09:37,2021-03-21 09:27:33,Glenwood Ave & Touhy Ave,Chicago Ave & Sheridan Rd,0,0 days 00:17:56,0.298889


# Conclusions and Final Thoughts
The Excel duplicate removal tool removed 27 duplicates, but it did not include the member_casual column in the helper column that the duplicate check list is emulating. The steps taken here show that one entry was removed in error in the Excel data cleaning. The number of duplicates in this file is low, at 0.011%, so I would consider the differences in clean up tools as negligible. 

The number of negative ride durations was not recorded, but the amount here is also negligible, relative to the total number of entries (0.0009%). While it's not recommended to just replace data values, I'm it for the exercise.

This was a good exercise in cleaning data using pandas, with my Excel process as a guiding hand. Now time to clean the rest of the data and analyze it all.